In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys

sys.path.append('../')
from Agents.A2CAgent_batch_v2 import Agent
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = Agent(n_actions=env.action_size(), input_dim=env.state_size(), alpha=1e-8, max_val=env.max_val())
bot0 = Bot()

wins = []

output_dir_a = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v6_batch_v2/0002/actor/weights_345300.hdf5'
output_dir_c = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v6_batch_v2/0002/critic/weights_345300.hdf5'

# output_dir_a = '../model_output/A2C_v5/0001/actor/weights_16000.hdf5'
# output_dir_c = '../model_output/A2C_v5/0001/critic/weights_16000.hdf5'

agent0.load_model(output_dir_a, output_dir_c)

for ep in tqdm(range(0, 10000)):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 0:
                    s_t = env.convert_state(player_turn)
                    action = agent0.act_test(s_t, action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
                s_, reward, game_over, player_turn_temp = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if episode_reward[0] > episode_reward[1]:
        wins.append(1)
    else:
        wins.append(0)

    if ep % 1000 == 0 and ep != 0:
        print(np.sum(wins))

2022-10-26 04:27:01.160679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 04:27:01.380842: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-26 04:27:02.628986: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-26 04:27:02.629109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

545


 20%|██        | 2001/10000 [07:27<30:18,  4.40it/s]

1097


 30%|███       | 3001/10000 [11:11<25:45,  4.53it/s]

...

 80%|████████  | 8001/10000 [29:51<07:37,  4.37it/s]

4445


 90%|█████████ | 9001/10000 [33:34<03:51,  4.31it/s]

5022


100%|██████████| 10000/10000 [37:18<00:00,  4.47it/s]


In [2]:
np.sum(wins)

5587